In [1]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import json

from glob import glob

import grl_tools

In [2]:
# Load Serap's icetop GRL files
grl_base = '/data/ana/CosmicRay/IceTop_GRL'
grl_files = sorted(glob(f'{grl_base}/IC86_20*.txt'))

it_goodruns = []
for grl_file in grl_files:
    it_goodruns += grl_tools.parse_seraps_GRL(grl_file)

In [3]:
# Load information from i3live
start = '2012-05-13 00:00:00'
end   = '2022-05-13 00:00:00'
i3_goodruns = grl_tools.i3live_grl(start, end, icetop=True)

In [4]:
# Calculate livetimes for icetop
i3_livetime = grl_tools.daily_livetime(i3_goodruns, it_goodruns)

10.2% of runs filtered out by IT GRL


In [7]:
# We need the counts for each day in the IceTop maps. Load the output from count_finder.py
prefix = '/data/user/fmcnally/icetop_12yr/stability'
count_files = sorted(glob(f'{prefix}/counts_20??_Tier?.json'))

# Load count files into a single dictionary
counts = {}
for tier in range(1, 5):
    counts[tier] = {}
    tier_files = [f for f in count_files if f'Tier{tier}' in f]
    for tier_file in tier_files:        
        with open(tier_file, 'r') as f:
            counts_i = json.load(f)
        counts[tier].update(counts_i)

CJ - at this point you have all the information you need:
1. `i3_livetime` is a dictionary with the format `i3_livetime[day][run] = time` (in seconds).
2. `counts` is a dictionary with the format `counts[day] = n_events`.

`i3_livetime` has more detail than you need, so you'll first want to produce something that looks like `livetime[day] = time` by summing up the individual run times for each day. Once you have that, you can make a plot by iterating over every day and plotting the rate (`counts[day]/livetime[day]`) vs the day